<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import nltk
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import classification_report

print("Text Results: ")

dataset1 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


countPos = 0
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][1]
    return 0

Data = []
Y = []

Data_test = []
Y_test = []
index = -1
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    if(val == 0 and countPos>38):
        continue
    Y.append(val)
    index+=1
    if(Y[index] == 0):
        countPos+=1

    
    fileName = "/content/drive/My Drive/MCA Dataset/train_data/" + str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    if(val == 0):
        continue
    Y.append(val)
    fileName = "/content/drive/My Drive/MCA Dataset/dev_data/" + str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])
    
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    fileName = "/content/drive/My Drive/MCA Dataset/test_data/" + str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])

    
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)
        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)

stop_words = set(stopwords.words('english'))


def remove_StopWOrds(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

def checkAcc(Y_pred, Y_test):
    correct = 0
    for i in range(len(Y_pred)):
        if(Y_pred[i] == Y_test[i]):
            correct+=1
    
    return float(correct)/len(Y_pred)


max_num_words = 15
max_num_sentence = 200

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words))
print(finalMatrix.shape)
for k in range(Data2.shape[0]):
    for i in range(min(max_num_sentence, len(Data2[k][0]))):
    	sentence = Data2[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X = np.zeros((len(Data2), max_num_sentence*max_num_words))
for i in range(len(finalMatrix)):
    X[i] = finalMatrix[i].flatten().reshape(1, -1)
 
#Test_data
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words))
print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    for i in range(min(max_num_sentence, len(Data2_test[k][0]))):
    	sentence = Data2_test[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix_test[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X_test = np.zeros((len(Data2_test), max_num_sentence*max_num_words))
for i in range(len(finalMatrix_test)):
    X_test[i] = finalMatrix_test[i].flatten().reshape(1, -1)

from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier()
clf1.fit(X,Y)
Y_pred_rf_1 = clf1.predict(X_test)
print("Random forests: ")
print(clf1.classification_report(Y_test,Y_pred_rf_1))
print("Train: "+ str(clf1.score(X,Y)))
print("Test: " + str(clf1.score(X_test,Y_test)))



from sklearn.svm import SVC

classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X, Y)

Y_pred_svm_1 = classifier.predict(X_test)
print("SVM: ")
print(classifier.classification_report(Y_test,Y_pred_svm_1))
print("Train: "+ str(classifier.score(X,Y)))
print("Test: " + str(classifier.score(X_test,Y_test)))

y_pred_train_svm_1 = classifier.predict(X)
y_pred_train_rf_1 = clf1.predict(X)


#Audio Features 
print("Audio Results: ")
import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    X = np.average(X, axis = 0)
    X = np.array(X.reshape(1, -1))
    return X
 

X_train = np.zeros((1,74))
Y_train = []

X_test = np.zeros((1,74))
#Y_test = []

for i in range(len(dataset2)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/test_data/'+str(int(dataset2[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_test = np.concatenate((X_test,X_temp),0)
    #Y_test.append(dataset2[i][1])
    

X_test = np.delete(X_test,0,0)
#Y_test = np.array(Y_test)
count_0 = 0
count_1 = 0

for i in range(len(dataset3)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/train_data/'+str(int(dataset3[i][0]))+"_COVAREP.csv",header = None)
    if(dataset3[i][1] == 0):
      count_0 +=1
      if(count_0<=39):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset3[i][1])
    else:
      count_1 +=1
      if(count_1<=28):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset3[i][1])

count_0 = 0
count_1 = 0    

for i in range(len(dataset1)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_data/'+str(int(dataset1[i][0]))+"_COVAREP.csv",header = None)
    if(dataset1[i][1] == 0):
      continue
    else:
      count_1+= 1
      if(count_1<=11):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset1[i][1])
    

X_train = np.delete(X_train,0,0)
Y_train = np.array(Y_train)
    
clf1 = RandomForestClassifier()
clf1.fit(X_train,Y_train)
Y_pred_rf_2 = clf1.predict(X_test)
print("Random forests: ")
print(clf1.classification_report(Y_test,Y_pred_rf_2))
print("Train: "+ str(clf1.score(X_train,Y_train)))
print("Test: " + str(clf1.score(X_test,Y_test)))


from sklearn.svm import SVC


classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred_svm_2 = classifier.predict(X_test)
print("SVM: ")
print(classifier.classification_report(Y_test,Y_pred_svm_2))
print("Train: "+ str(classifier.score(X_train,Y_train)))
print("Test: " + str(classifier.score(X_test,Y_test)))

y_pred_train_svm_2 = classifier.predict(X_train)
y_pred_train_rf_2 = clf1.predict(X_train)


#Video

print("Video Results: ")

from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

def processData(data):
    # print(data)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))

    X = np.mean(X, 0)
    X = np.array(X.reshape(1, -1))
    # print(X)
    # print(X.shape)
    return X

def getData(filename):
    # print(filename)
    data = pd.read_csv(filename,delimiter=',')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    # X5 = getData(file5)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    # print(X)
    # print(X.shape)
    return X

count0 = 0
count1 = 0

def makeDataset(location, folder):
    global count0
    global count1
    file  = np.array(pd.read_csv(location,delimiter=',',encoding='utf-8'))[:, 0:2]
    X_temp = np.zeros((1,388))
    Y_temp = []
    for i in range(len(file)):
      if(int(file[i][1]) == 0):
        count0 += 1
        if(count0 >39):
          continue
      patientID = (str(int(file[i][0])))
      #print(patientID)
      string = '/content/drive/My Drive/MCA Dataset/' + folder +"/"+ patientID
      X_temp = np.concatenate((X_temp, makeDataPoint(string)), 0)
      Y_temp.append(int(file[i][1]))
    X_temp = np.delete(X_temp,0,0)
    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp

print("train")
X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')
print("dev")
X_dev, Y_dev = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')
count0 = 0
X_train = np.concatenate((X_train, X_dev), 0  )
Y_train = np.concatenate((Y_train, Y_dev), 0  )

print("test")
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')

print(Y_train)

clf1 = RandomForestClassifier()
clf1.fit(X_train,Y_train)
Y_pred_rf_3 = clf1.predict(X_test)
print("Random forests: ")
print(clf1.classification_report(Y_test,Y_pred_rf_3))
print("Train: "+ str(clf1.score(X_train,Y_train)))
print("Test: " + str(clf1.score(X_test,Y_test)))

from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred_svm_3 = classifier.predict(X_test)
print("SVM: ")
print(classifier.classification_report(Y_test,Y_pred_svm_3))
print("Train: "+ str(classifier.score(X_train,Y_train)))
print("Test: " + str(classifier.score(X_test,Y_test)))

y_pred_train_svm_3 = classifier.predict(X_train)
y_pred_train_rf_3 = clf1.predict(X_train)


#Combine 
# X_2 = np.column_stack((y_pred_train1,y_pred_train2,y_pred_train3))

# from sklearn.svm import SVC
# classifier2 = SVC(kernel = 'rbf', random_state = 0, gamma = 'auto')
# classifier2.fit(X_2, Y_train)
# y_pred_combine = classifier.predict(np.column_stack((y_pred1, y_pred2,y_pred3)))
# print(classification_report(Y_test, y_pred_combine))


AttributeError: ignored

In [0]:
!pip install nltk

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449904 sha256=1c8e148ae9ba0382bdb199ace791325c23068717235aa336f606b7cd0ef52a34
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [0]:
import nltk

AttributeError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
